In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [256, 256]  # 单层LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 1  # 二分类设置为1，多分类设置为类别的数目
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self.config = config
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self.labelList = []
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        
        if self.config.numClasses == 1:
            labels = df["sentiment"].tolist()
        elif self.config.numClasses > 1:
            labels = df["rate"].tolist()
            
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels
    
    def _labelToIndex(self, labels, label2idx):
        """
        将标签转换成索引表示
        """
        labelIds = [label2idx[label] for label in labels]
        return labelIds
    
    def _wordToIndex(self, reviews, word2idx):
        """
        将词转换成索引
        """
        reviewIds = [[word2idx.get(item, word2idx["UNK"]) for item in review] for review in reviews]
        return reviewIds
        
    def _genTrainEvalData(self, x, y, word2idx, rate):
        """
        生成训练集和验证集
        """
        reviews = []
        for review in x:
            if len(review) >= self._sequenceLength:
                reviews.append(review[:self._sequenceLength])
            else:
                reviews.append(review + [word2idx["PAD"]] * (self._sequenceLength - len(review)))
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(y[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(y[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews, labels):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        word2idx = dict(zip(vocab, list(range(len(vocab)))))
        
        uniqueLabel = list(set(labels))
        label2idx = dict(zip(uniqueLabel, list(range(len(uniqueLabel)))))
        self.labelList = list(range(len(uniqueLabel)))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/word2idx.json", "w", encoding="utf-8") as f:
            json.dump(word2idx, f)
        
        with open("../data/wordJson/label2idx.json", "w", encoding="utf-8") as f:
            json.dump(label2idx, f)
        
        return word2idx, label2idx
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("PAD")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        word2idx, label2idx = self._genVocabulary(reviews, labels)
        
        # 将标签和句子数值化
        labelIds = self._labelToIndex(labels, label2idx)
        reviewIds = self._wordToIndex(reviews, word2idx)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviewIds, labelIds, word2idx, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

youre不存在于词向量中
youll不存在于词向量中
theyre不存在于词向量中
youve不存在于词向量中
werent不存在于词向量中
youd不存在于词向量中
hasnt不存在于词向量中
shouldnt不存在于词向量中
weve不存在于词向量中
theyve不存在于词向量中
1010不存在于词向量中
wouldve不存在于词向量中
hed不存在于词向量中
andor不存在于词向量中
couldve不存在于词向量中
810不存在于词向量中
itthe不存在于词向量中
710不存在于词向量中
theyd不存在于词向量中
writerdirector不存在于词向量中
moviei不存在于词向量中
iti不存在于词向量中
theyll不存在于词向量中
310不存在于词向量中
910不存在于词向量中
410不存在于词向量中
itll不存在于词向量中
lees不存在于词向量中
familys不存在于词向量中
disneys不存在于词向量中
filmi不存在于词向量中
210不存在于词向量中
shouldve不存在于词向量中
*12不存在于词向量中
shakespeares不存在于词向量中
hitlers不存在于词向量中
freddys不存在于词向量中
brazil:不存在于词向量中
fords不存在于词向量中
storys不存在于词向量中
ohara不存在于词向量中
stewarts不存在于词向量中
kellys不存在于词向量中
itthis不存在于词向量中
tonys不存在于词向量中
scotts不存在于词向量中
keatons不存在于词向量中
rosemarys不存在于词向量中
ches不存在于词向量中
timethe不存在于词向量中
themthe不存在于词向量中
branaghs不存在于词向量中
mustve不存在于词向量中
kubricks不存在于词向量中
hitchcocks不存在于词向量中
lynchs不存在于词向量中
bakshis不存在于词向量中
allthe不存在于词向量中
whove不存在于词向量中
whod不存在于词向量中
itit不存在于词向量中
smiths不存在于词向量中
obrien不存在于词向量中
cravens不存在于词向量中
palmas不存在于词向量中
timei不存在于词向量中
movieit不存在于词向量中
waythe不

bagman不存在于词向量中
sm不存在于词向量中
bloss不存在于词向量中
timea不存在于词向量中
blithe不存在于词向量中
allendes不存在于词向量中
worki不存在于词向量中
chers不存在于词向量中
upi不存在于词向量中
himher不存在于词向量中
bei不存在于词向量中
baldwins不存在于词向量中
mayors不存在于词向量中
pythons不存在于词向量中
walshs不存在于词向量中
lius不存在于词向量中
spocks不存在于词向量中
knowi不存在于词向量中
youit不存在于词向量中
frankensteins不存在于词向量中
reids不存在于词向量中
throughi不存在于词向量中
okay!不存在于词向量中
wasis不存在于词向量中
happenthe不存在于词向量中
facethe不存在于词向量中
pickfords不存在于词向量中
barrymores不存在于词向量中
yaara不存在于词向量中
walkin不存在于词向量中
julians不存在于词向量中
aroundthe不存在于词向量中
$100000不存在于词向量中
differentthe不存在于词向量中
orwells不存在于词向量中
itfor不存在于词向量中
mankinds不存在于词向量中
crosbys不存在于词向量中
starsthe不存在于词向量中
gillians不存在于词向量中
labours不存在于词向量中
videodvd不存在于词向量中
timeits不存在于词向量中
alienator不存在于词向量中
timeif不存在于词向量中
toothis不存在于词向量中
tierneys不存在于词向量中
sants不存在于词向量中
graysons不存在于词向量中
redgraves不存在于词向量中
tronje不存在于词向量中
somethingi不存在于词向量中
movieall不存在于词向量中
prag不存在于词向量中
caesars不存在于词向量中
moviedo不存在于词向量中
therere不存在于词向量中
christies不存在于词向量中
thingthe不存在于词向量中
matata不存在于词向量中
voights不存在于词向量中
brendans不存在于词向量中
grays不存在于词向量中
hamlet

**spoilers**不存在于词向量中
aboriginals不存在于词向量中
alexandres不存在于词向量中
emigrant不存在于词向量中
paramilitaries不存在于词向量中
itof不存在于词向量中
snore-fest不存在于词向量中
lettieri不存在于词向量中
gulfax不存在于词向量中
blackwhite不存在于词向量中
werewolfs不存在于词向量中
filmto不存在于词向量中
marjories不存在于词向量中
hugsy不存在于词向量中
zuckerman不存在于词向量中
coote不存在于词向量中
watchtower不存在于词向量中
hereafter不存在于词向量中
hyser不存在于词向量中
tysons不存在于词向量中
site!不存在于词向量中
perspectivethe不存在于词向量中
thunderbolt不存在于词向量中
potentialthe不存在于词向量中
everythingthe不存在于词向量中
abstractions不存在于词向量中
rainbeaux不存在于词向量中
heorot不存在于词向量中
wtf!不存在于词向量中
monteith不存在于词向量中
lukass不存在于词向量中
lancasters不存在于词向量中
4-bad不存在于词向量中
richies不存在于词向量中
rothschild不存在于词向量中
lengles不存在于词向量中
wangs不存在于词向量中
noland不存在于词向量中
mechs不存在于词向量中
crouses不存在于词向量中
så不存在于词向量中
inhe不存在于词向量中
videothe不存在于词向量中
friendless不存在于词向量中
antonella不存在于词向量中
hammerhead:不存在于词向量中
writerthe不存在于词向量中
acs不存在于词向量中
fixit不存在于词向量中
directionthe不存在于词向量中
`mr不存在于词向量中
pvt不存在于词向量中
tgif不存在于词向量中
valcos不存在于词向量中
holms不存在于词向量中
buddys不存在于词向量中
twos不存在于词向量中
domini不存在于词向量中
trombone不存在于词向量中
showand不存在于词向量中
demonst

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000,)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 构建模型
class BiLSTM(object):
    """
    Bi-LSTM 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.int32, [None], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):

            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2]
                    self.embeddedWords = tf.concat(outputs, 2)
        
        # 去除最后时间步的输出作为全连接的输入
        finalOutput = self.embeddedWords[:, -1, :]
        
        outputSize = config.model.hiddenSizes[-1] * 2  # 因为是双向LSTM，最终的输出值是fw和bw的拼接，因此要乘以2
        output = tf.reshape(finalOutput, [-1, outputSize])  # reshape成全连接层的输入维度
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, config.numClasses],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[config.numClasses]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.logits = tf.nn.xw_plus_b(output, outputW, outputB, name="logits")
            if config.numClasses == 1:
                self.predictions = tf.cast(tf.greater_equal(self.logits, 0.0), tf.float32, name="predictions")
            elif config.numClasses > 1:
                self.predictions = tf.argmax(self.logits, axis=-1, name="predictions")
            print(self.predictions)
            
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            if config.numClasses == 1:
                losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, labels=tf.cast(tf.reshape(self.inputY, [-1, 1]), 
                                                                                                    dtype=tf.float32))
            elif config.numClasses > 1:
                losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.inputY)
                
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [7]:
"""
定义各类性能指标
"""

def mean(item: list) -> float:
    """
    计算列表中元素的平均值
    :param item: 列表对象
    :return:
    """
    res = sum(item) / len(item) if len(item) > 0 else 0
    return res


def accuracy(pred_y, true_y):
    """
    计算二类和多类的准确率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]
    corr = 0
    for i in range(len(pred_y)):
        if pred_y[i] == true_y[i]:
            corr += 1
    acc = corr / len(pred_y) if len(pred_y) > 0 else 0
    return acc


def binary_precision(pred_y, true_y, positive=1):
    """
    二类的精确率计算
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param positive: 正例的索引表示
    :return:
    """
    corr = 0
    pred_corr = 0
    for i in range(len(pred_y)):
        if pred_y[i] == positive:
            pred_corr += 1
            if pred_y[i] == true_y[i]:
                corr += 1

    prec = corr / pred_corr if pred_corr > 0 else 0
    return prec


def binary_recall(pred_y, true_y, positive=1):
    """
    二类的召回率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param positive: 正例的索引表示
    :return:
    """
    corr = 0
    true_corr = 0
    for i in range(len(pred_y)):
        if true_y[i] == positive:
            true_corr += 1
            if pred_y[i] == true_y[i]:
                corr += 1

    rec = corr / true_corr if true_corr > 0 else 0
    return rec


def binary_f_beta(pred_y, true_y, beta=1.0, positive=1):
    """
    二类的f beta值
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param beta: beta值
    :param positive: 正例的索引表示
    :return:
    """
    precision = binary_precision(pred_y, true_y, positive)
    recall = binary_recall(pred_y, true_y, positive)
    try:
        f_b = (1 + beta * beta) * precision * recall / (beta * beta * precision + recall)
    except:
        f_b = 0
    return f_b


def multi_precision(pred_y, true_y, labels):
    """
    多类的精确率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param labels: 标签列表
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]

    precisions = [binary_precision(pred_y, true_y, label) for label in labels]
    prec = mean(precisions)
    return prec


def multi_recall(pred_y, true_y, labels):
    """
    多类的召回率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param labels: 标签列表
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]

    recalls = [binary_recall(pred_y, true_y, label) for label in labels]
    rec = mean(recalls)
    return rec


def multi_f_beta(pred_y, true_y, labels, beta=1.0):
    """
    多类的f beta值
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param labels: 标签列表
    :param beta: beta值
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]

    f_betas = [binary_f_beta(pred_y, true_y, beta, label) for label in labels]
    f_beta = mean(f_betas)
    return f_beta


def get_binary_metrics(pred_y, true_y, f_beta=1.0):
    """
    得到二分类的性能指标
    :param pred_y:
    :param true_y:
    :param f_beta:
    :return:
    """
    acc = accuracy(pred_y, true_y)
    recall = binary_recall(pred_y, true_y)
    precision = binary_precision(pred_y, true_y)
    f_beta = binary_f_beta(pred_y, true_y, f_beta)
    return acc, recall, precision, f_beta


def get_multi_metrics(pred_y, true_y, labels, f_beta=1.0):
    """
    得到多分类的性能指标
    :param pred_y:
    :param true_y:
    :param labels:
    :param f_beta:
    :return:
    """
    acc = accuracy(pred_y, true_y)
    recall = multi_recall(pred_y, true_y, labels)
    precision = multi_precision(pred_y, true_y, labels)
    f_beta = multi_f_beta(pred_y, true_y, labels, f_beta)
    return acc, recall, precision, f_beta

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding
labelList = data.labelList

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.6  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTM(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        savedModelPath = "../model/Bi-LSTM/savedModel"
        if os.path.exists(savedModelPath):
            os.rmdir(savedModelPath)
        builder = tf.saved_model.builder.SavedModelBuilder(savedModelPath)
            
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions],
                feed_dict)
            
            timeStr = datetime.datetime.now().isoformat()
            
            if config.numClasses == 1:
                acc, recall, prec, f_beta = get_binary_metrics(pred_y=predictions, true_y=batchY)
                
            elif config.numClasses > 1:
                acc, recall, prec, f_beta = get_multi_metrics(pred_y=predictions, true_y=batchY,
                                                              labels=labelList)
                
            trainSummaryWriter.add_summary(summary, step)
            
            return loss, acc, prec, recall, f_beta

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions],
                feed_dict)
            
            if config.numClasses == 1:
            
                acc, precision, recall, f_beta = get_binary_metrics(pred_y=predictions, true_y=batchY)
            elif config.numClasses > 1:
                acc, precision, recall, f_beta = get_multi_metrics(pred_y=predictions, true_y=batchY, labels=labelList)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, precision, recall, f_beta
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                loss, acc, prec, recall, f_beta = trainStep(batchTrain[0], batchTrain[1])
                
                currentStep = tf.train.global_step(sess, globalStep) 
                print("train: step: {}, loss: {}, acc: {}, recall: {}, precision: {}, f_beta: {}".format(
                    currentStep, loss, acc, recall, prec, f_beta))
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    f_betas = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, precision, recall, f_beta = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        f_betas.append(f_beta)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {},precision: {}, recall: {}, f_beta: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(precisions),
                                                                                                       mean(recalls), mean(f_betas)))
                    
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

        outputs = {"predictions": tf.saved_model.utils.build_tensor_info(lstm.predictions)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

        builder.save()

W0726 14:25:57.413639 140504067012352 deprecation.py:323] From <ipython-input-6-4ca767c2c1b3>:31: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0726 14:25:57.420497 140504067012352 deprecation.py:323] From <ipython-input-6-4ca767c2c1b3>:43: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
W0726 14:25:57.421681 140504067012352 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/rnn.py:464: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent t

Tensor("output/predictions:0", shape=(?, 1), dtype=float32)
Writing to /home/jqxx/jupyternotebook/textClassifier-master/Bi-LSTM/summarys

start training model
train: step: 1, loss: 0.691848635673523, acc: 0.5234375, recall: 0.9523809523809523, precision: 0.5084745762711864, f_beta: 0.6629834254143646
train: step: 2, loss: 0.7310537099838257, acc: 0.453125, recall: 1.0, precision: 0.44881889763779526, f_beta: 0.6195652173913043
train: step: 3, loss: 0.8086796998977661, acc: 0.53125, recall: 0.4225352112676056, precision: 0.6122448979591837, f_beta: 0.4999999999999999
train: step: 4, loss: 0.7384079098701477, acc: 0.4609375, recall: 0.42424242424242425, precision: 0.4745762711864407, f_beta: 0.448
train: step: 5, loss: 0.7022448182106018, acc: 0.5546875, recall: 0.9855072463768116, precision: 0.5483870967741935, f_beta: 0.7046632124352332
train: step: 6, loss: 0.7710385322570801, acc: 0.484375, recall: 1.0, precision: 0.48031496062992124, f_beta: 0.648936170212766
train: step: 7, loss: 0

train: step: 59, loss: 0.6935716867446899, acc: 0.515625, recall: 0.8307692307692308, precision: 0.5142857142857142, f_beta: 0.6352941176470588
train: step: 60, loss: 0.6963160634040833, acc: 0.5078125, recall: 0.734375, precision: 0.5053763440860215, f_beta: 0.5987261146496815
train: step: 61, loss: 0.7015841007232666, acc: 0.515625, recall: 0.6133333333333333, precision: 0.5822784810126582, f_beta: 0.5974025974025974
train: step: 62, loss: 0.69252610206604, acc: 0.46875, recall: 0.7894736842105263, precision: 0.44554455445544555, f_beta: 0.569620253164557
train: step: 63, loss: 0.7003044486045837, acc: 0.4453125, recall: 0.6515151515151515, precision: 0.4725274725274725, f_beta: 0.5477707006369427
train: step: 64, loss: 0.6912063956260681, acc: 0.5390625, recall: 0.7941176470588235, precision: 0.5454545454545454, f_beta: 0.6467065868263473
train: step: 65, loss: 0.7022831439971924, acc: 0.4609375, recall: 0.7868852459016393, precision: 0.46153846153846156, f_beta: 0.5818181818181818


train: step: 117, loss: 0.6259950399398804, acc: 0.6640625, recall: 0.9393939393939394, precision: 0.6138613861386139, f_beta: 0.7425149700598802
train: step: 118, loss: 0.5430752635002136, acc: 0.796875, recall: 0.7285714285714285, precision: 0.8793103448275862, f_beta: 0.796875
train: step: 119, loss: 0.486763060092926, acc: 0.8203125, recall: 0.8888888888888888, precision: 0.810126582278481, f_beta: 0.847682119205298
train: step: 120, loss: 0.6891999244689941, acc: 0.671875, recall: 0.7538461538461538, precision: 0.6533333333333333, f_beta: 0.7
train: step: 121, loss: 0.553303599357605, acc: 0.75, recall: 0.5357142857142857, precision: 0.8333333333333334, f_beta: 0.6521739130434783
train: step: 122, loss: 0.664178729057312, acc: 0.625, recall: 0.24193548387096775, precision: 0.9375, f_beta: 0.3846153846153846
train: step: 123, loss: 0.6045092344284058, acc: 0.65625, recall: 0.390625, precision: 0.8333333333333334, f_beta: 0.5319148936170213
train: step: 124, loss: 0.627227783203125,

train: step: 175, loss: 0.7383542656898499, acc: 0.5, recall: 0.03225806451612903, precision: 0.3333333333333333, f_beta: 0.0588235294117647
train: step: 176, loss: 0.6970446109771729, acc: 0.5390625, recall: 0.2923076923076923, precision: 0.59375, f_beta: 0.3917525773195876
train: step: 177, loss: 0.7181582450866699, acc: 0.5234375, recall: 0.6545454545454545, precision: 0.46153846153846156, f_beta: 0.5413533834586467
train: step: 178, loss: 0.6923552751541138, acc: 0.5390625, recall: 0.75, precision: 0.5274725274725275, f_beta: 0.6193548387096776
train: step: 179, loss: 0.7477456331253052, acc: 0.46875, recall: 0.6612903225806451, precision: 0.4659090909090909, f_beta: 0.5466666666666665
train: step: 180, loss: 0.7157608270645142, acc: 0.515625, recall: 0.7777777777777778, precision: 0.5051546391752577, f_beta: 0.6125
train: step: 181, loss: 0.7165135145187378, acc: 0.4609375, recall: 0.8, precision: 0.43137254901960786, f_beta: 0.5605095541401275
train: step: 182, loss: 0.6704245805

...

train: step: 1419, loss: 0.019388094544410706, acc: 0.9921875, recall: 1.0, precision: 0.9841269841269841, f_beta: 0.9919999999999999
train: step: 1420, loss: 0.08012446761131287, acc: 0.984375, recall: 0.9863013698630136, precision: 0.9863013698630136, f_beta: 0.9863013698630136
train: step: 1421, loss: 0.07947368174791336, acc: 0.984375, recall: 0.9710144927536232, precision: 1.0, f_beta: 0.9852941176470589
train: step: 1422, loss: 0.008680363185703754, acc: 1.0, recall: 1.0, precision: 1.0, f_beta: 1.0
train: step: 1423, loss: 0.00938320904970169, acc: 1.0, recall: 1.0, precision: 1.0, f_beta: 1.0
train: step: 1424, loss: 0.05768290162086487, acc: 0.9921875, recall: 1.0, precision: 0.9850746268656716, f_beta: 0.9924812030075187
train: step: 1425, loss: 0.04898009076714516, acc: 0.9921875, recall: 1.0, precision: 0.984375, f_beta: 0.9921259842519685
train: step: 1426, loss: 0.04773065820336342, acc: 0.9921875, recall: 1.0, precision: 0.9852941176470589, f_beta: 0.9925925925925926
tra

train: step: 1484, loss: 0.16163915395736694, acc: 0.96875, recall: 0.9402985074626866, precision: 1.0, f_beta: 0.9692307692307692
train: step: 1485, loss: 0.06388534605503082, acc: 0.984375, recall: 0.9850746268656716, precision: 0.9850746268656716, f_beta: 0.9850746268656716
train: step: 1486, loss: 0.08801238238811493, acc: 0.9765625, recall: 0.9682539682539683, precision: 0.9838709677419355, f_beta: 0.976
train: step: 1487, loss: 0.08367089927196503, acc: 0.984375, recall: 0.9827586206896551, precision: 0.9827586206896551, f_beta: 0.9827586206896551
train: step: 1488, loss: 0.009471505880355835, acc: 1.0, recall: 1.0, precision: 1.0, f_beta: 1.0
train: step: 1489, loss: 0.04764075204730034, acc: 0.9921875, recall: 0.9830508474576272, precision: 1.0, f_beta: 0.9914529914529915
train: step: 1490, loss: 0.08743016421794891, acc: 0.984375, recall: 1.0, precision: 0.9636363636363636, f_beta: 0.9814814814814815
train: step: 1491, loss: 0.07442249357700348, acc: 0.984375, recall: 0.966101

train: step: 1547, loss: 0.12703736126422882, acc: 0.9765625, recall: 0.96875, precision: 0.9841269841269841, f_beta: 0.9763779527559054
train: step: 1548, loss: 0.08478166162967682, acc: 0.9765625, recall: 0.9710144927536232, precision: 0.9852941176470589, f_beta: 0.9781021897810219
train: step: 1549, loss: 0.05510714650154114, acc: 0.984375, recall: 1.0, precision: 0.9692307692307692, f_beta: 0.9843749999999999
train: step: 1550, loss: 0.007434182800352573, acc: 1.0, recall: 1.0, precision: 1.0, f_beta: 1.0
train: step: 1551, loss: 0.023899704217910767, acc: 0.9921875, recall: 1.0, precision: 0.9824561403508771, f_beta: 0.9911504424778761
train: step: 1552, loss: 0.011967000551521778, acc: 1.0, recall: 1.0, precision: 1.0, f_beta: 1.0
train: step: 1553, loss: 0.08131159096956253, acc: 0.984375, recall: 1.0, precision: 0.9682539682539683, f_beta: 0.9838709677419354
train: step: 1554, loss: 0.031063828617334366, acc: 0.984375, recall: 0.9859154929577465, precision: 0.9859154929577465, 

W0726 15:25:22.224742 140504067012352 deprecation.py:323] From <ipython-input-8-df1e8dbf2db4>:149: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0726 15:25:22.229662 140504067012352 deprecation.py:506] From <ipython-input-8-df1e8dbf2db4>:158: calling SavedModelBuilder.add_meta_graph_and_variables (from tensorflow.python.saved_model.builder_impl) with legacy_init_op is deprecated and will be removed in a future version.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.


train: step: 1560, loss: 0.032786302268505096, acc: 0.9921875, recall: 0.9859154929577465, precision: 1.0, f_beta: 0.9929078014184397


In [12]:
x = "this movie is full of references like mad max ii the wild one and many others the ladybug´s face it´s a clear reference or tribute to peter lorre this movie is a masterpiece we´ll talk much more about in the future"

# 注：下面两个词典要保证和当前加载的模型对应的词典是一致的
with open("../data/wordJson/word2idx.json", "r", encoding="utf-8") as f:
    word2idx = json.load(f)
        
with open("../data/wordJson/label2idx.json", "r", encoding="utf-8") as f:
    label2idx = json.load(f)
idx2label = {value: key for key, value in label2idx.items()}
    
xIds = [word2idx.get(item, word2idx["UNK"]) for item in x.split(" ")]
if len(xIds) >= config.sequenceLength:
    xIds = xIds[:config.sequenceLength]
else:
    xIds = xIds + [word2idx["PAD"]] * (config.sequenceLength - len(xIds))

graph = tf.Graph()
with graph.as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False, gpu_options=gpu_options)
    sess = tf.Session(config=session_conf)

    with sess.as_default():
        checkpoint_file = tf.train.latest_checkpoint("../model/Bi-LSTM/model/")
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # 获得需要喂给模型的参数，输出的结果依赖的输入值
        inputX = graph.get_operation_by_name("inputX").outputs[0]
        dropoutKeepProb = graph.get_operation_by_name("dropoutKeepProb").outputs[0]

        # 获得输出的结果
        predictions = graph.get_tensor_by_name("output/predictions:0")

        pred = sess.run(predictions, feed_dict={inputX: [xIds], dropoutKeepProb: 1.0})[0]
        
pred = [idx2label[item] for item in pred]     
print(pred)

['1']
